In [ ]:
import pennylane as qml
from pennylane import numpy as np


def construct_estimation_qnode(U, eigvec, n_estimation_wires):
    """Construct and return a QNode that will perform QPE with a given number
    of estimation wires.

    Use the QuantumPhaseEstimation template.

    Args:
        U (array[complex]): A random 1-qubit or 2-qubit unitary matrix.
        eigvec (array[complex]): An eigenvector whose eigenvalue we are trying to find.
        n_estimation_wires (int): The number of estimation wires to use.

    Returns:
        QNode: A QNode that performs QPE for U and the given eigenvector
        and uses the provided number of estimation wires.
    """
    # YOUR CODE HERE
    from pennylane.templates import QuantumPhaseEstimation
    
    n_target_wires = 1 if len(U) == 2 else 2
    estimation_wires = range(n_target_wires, n_estimation_wires + n_target_wires)
    dev = qml.device("default.qubit", wires=n_estimation_wires + n_target_wires)
    target_wires = [i for i in range(n_target_wires)]
    
    @qml.qnode(dev)
    def qnode():
        """Your QNode should not take any parameters as input."""
        qml.QubitStateVector(eigvec, wires=target_wires)
        QuantumPhaseEstimation(
            U,
            estimation_wires=estimation_wires,
            target_wires=target_wires,
        )
    
        return qml.probs(estimation_wires)

    return qnode


def compute_estimation_window(U, eigvec):
    """Use QPE to compute a phase window that bounds the value of a phase
    that cannot be expressed exactly using t bits.

    Suppose the true value of the phase is exp(2 * np.pi * 1j * theta).  The
    phase window should be an array of two real numbers [theta_lower,
    theta_upper], such that the interval between them contains the true value of
    theta. The width of the window should be no larger than 1e-3; you will need
    to determine the number of estimation wires required to obtain a window to
    this precision.

    Your code will be tested on 1-qubit unitaries, and 2-qubit unitaries.

    Args:
        U (array[complex]): A random 1-qubit or 2-qubit unitary matrix.
        eigvec (array[complex]): An eigenvector whose eigenvalue we are trying to find.

    Returns:
        array[float]: An array representing the lower and upper bound of the
        value of the phase we are estimating (the absolute difference between
        the two values should be less than 1e-3).
    """
    phase_window = [0.0, 0.0]

    # YOUR CODE HERE
    
    ## taken from Xanadu P3.1
    def phase_window(probs, estimation_wires):
        arr = []
        increment = 2 ** -len(estimation_wires)
        for i in range(2 ** len(estimation_wires)):
            arr.append(i * increment)
        probs = [float(i) for i in probs]
        values = sorted(probs)
        bound_1 = arr[probs.index(values[-1])] # MOST LIKELY OUTCOME
        bound_2 = arr[probs.index(values[-2])] # SECOND MOST LIKELY OUTCOME
        return (bound_1, bound_2)
    
    window_width = np.Inf
    n_estimators = 2
    while (window_width > 1e-3):
        estimation_wires = [j for j in range(n_estimators)]
        qpe = construct_estimation_qnode(U, eigvec, n_estimators)
        pw = phase_window(qpe(), estimation_wires)
        window_width = np.abs(pw[0]-pw[1])
        n_estimators += 1

    return pw